In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd

In [ ]:
local_user = 'root'
password = '*******'
local_host = 'localhost'
database = 'spotify_tracker_schema'
AWS_user = 'fTAVARES'
AWS_host = 'spotifyapp-database.c0niqo2yq4lw.us-east-1.rds.amazonaws.com'

def get_connection(user,password,host, database):
    # Create a connection to the MySQL database using SQLAlchemy
    return create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, 3306,database)
        )

AWS_connection = get_connection(AWS_user, password, AWS_host, database)
local_connection = get_connection(local_user, password, local_host, database)

In [30]:
query_f_played = 'SELECT * FROM f_played'
query_d_songs = 'SELECT * FROM d_songs'
query_d_artists = 'SELECT * FROM d_artists'
query_d_users = 'SELECT * FROM d_user'
query_d_songs_artists = 'SELECT * FROM d_songs_artists'
query_insert_f_played = """ INSERT IGNORE INTO f_played (song_id, song_name, user_id, timestamp, duration, dt_listened)
                    VALUES (:song_id, :song_name, :user_id, :timestamp, :duration, :dt_listened)"""
query_insert_d_user = """
                INSERT IGNORE INTO d_user (user_id, user_name, email, country, membership, user_image)
                VALUES (:user_id, :user_name, :email, :country, :membership, :user_image)
            """
query_insert_d_songs = """
            INSERT IGNORE INTO d_songs (song_id, song_name, duration, album_name, song_image, song_url, song_uri, album_url, album_id)
            VALUES (:song_id, :song_name, :duration, :album_name, :song_image, :song_url, :song_uri, :album_url, :album_id)"""
query_insert_d_songs_artists = """
                INSERT IGNORE INTO d_songs_artists (song_id, artist_id)
                VALUES (:song_id, :artist_id)
            """
query_insert_d_artists = """
            INSERT IGNORE INTO d_artists (artist_id, artist_name, artist_url, artist_uri, artist_image)
            VALUES (:artist_id, :artist_name, :artist_url, :artist_uri, :artist_image)
            """

queries = {
    'f_played': query_f_played,
    'd_songs': query_d_songs,
    'd_artists': query_d_artists,
    'd_user': query_d_users,
    'd_songs_artists': query_d_songs_artists,
    'insert_f_played': query_insert_f_played,
    'insert_d_user': query_insert_d_user,
    'insert_d_songs': query_insert_d_songs,
    'insert_d_songs_artists': query_insert_d_songs_artists,
    'insert_d_artists': query_insert_d_artists}

In [31]:
tables = ['d_songs', 'd_artists', 'd_user', 'd_songs_artists','f_played', 'insert_f_played',  'insert_d_songs_artists','insert_d_user', 'insert_d_artists','insert_d_songs']
L = 0
R = len(tables) - 1
while L < R:
    with AWS_connection.begin() as connection:
        if tables[L] != 'f_played':
            print(f'Inserting data from {tables[L]} to {tables[R]}')
            data = pd.read_sql(queries[tables[L]], local_connection)
            connection.execute(text(queries[tables[R]]), data.to_dict(orient='records'))
        else:
            print(f'Inserting data from {tables[L]} to {tables[R]} in chunks')
            for chunk in pd.read_sql(queries[tables[L]], local_connection, chunksize=30000):
                connection.execute(text(queries[tables[R]]), chunk.to_dict(orient='records'))
    L += 1
    R -= 1
        



Inserting data from d_songs to insert_d_songs
Inserting data from d_artists to insert_d_artists
Inserting data from d_user to insert_d_user
Inserting data from d_songs_artists to insert_d_songs_artists
Inserting data from f_played to insert_f_played in chunks
